In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import datetime
from datetime import timedelta
from scipy.stats import randint, uniform
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from math import sqrt
seed = 342
np.random.seed(seed)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

C:\Users\arthu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.columns

Index(['id', 'name', 'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'country_code', 'country', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_ch

In [5]:
(train.isnull().sum()/train.shape[0]).sort_values()

id                                  0.000000
neighbourhood_group_cleansed        0.000000
calculated_host_listings_count      0.000000
country                             0.000000
property_type                       0.000000
room_type                           0.000000
accommodates                        0.000000
bed_type                            0.000000
amenities                           0.000000
price                               0.000000
guests_included                     0.000000
extra_people                        0.000000
minimum_nights                      0.000000
maximum_nights                      0.000000
number_of_reviews                   0.000000
instant_bookable                    0.000000
is_business_travel_ready            0.000000
cancellation_policy                 0.000000
require_guest_profile_picture       0.000000
require_guest_phone_verification    0.000000
neighbourhood_cleansed              0.000000
host_verifications                  0.000000
country_co

In [6]:
train.head()

,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,22267382,Modern and Cozy Large Studio in Brooklyn,Modern large studio with new amenities and app...,Our place is a little quiet sanctuary in the h...,Modern large studio with new amenities and app...,none,"BAM, Barclays, Brooklyn City Point, Fort Green...",NaN,"Subway: 2,3,4,5,A,C,B,Q,G",Washer/Dryer Dishwasher Internet Gym Roof Top ...,"Depending on the time of your visit, I'll be h...","- Please be respectful of our neighbors, no lo...",1910170,Katarina,2012-03-12,"Los Angeles, California, United States","Traveler, Artist, Philanthropist",NaN,NaN,NaN,f,Fort Greene,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Fort Greene,Brooklyn,Brooklyn,NY,11217,New York,US,United States,Loft,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Gym,Breakfast...",NaN,145.0,2,$30.00,7,12,6,2018-01-01,2018-07-03,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,f,flexible,f,f,1,0.59
1,2473861,Royal Harlem TRIPLEX Home 5 Beds,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,none,HARLEM is a piece of real NY history overflowi...,HARLEM RESTAURANTS Red Rooster Harlem -- excel...,PUBLIC TRANSPORTATION: Conveniently near all p...,The WHOLE ENTIRE HOUSE,NaN,"Smoking, pets and unaccounted guests NOT permi...",8826175,Grover,2013-09-14,"New York, New York, United States",As the owners of the property we have realized...,within an hour,100%,NaN,f,Mid-Wilshire,4.0,"['email', 'phone', 'reviews', 'manual_offline']",t,t,Harlem,Manhattan,New York,NY,10027,New York,US,United States,House,Entire home/apt,8,1.0,3.0,5.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,175.0,3,$19.00,3,30,137,2014-04-15,2018-10-20,91.0,9.0,9.0,9.0,9.0,9.0,9.0,t,f,moderate,f,f,3,2.47
2,25079703,Sunny East Village Studio,"Clean, hip and well designed sun drenched East...",This is a rare East Village studio with it's h...,"Clean, hip and well designed sun drenched East...",none,East Village is one of the last remaining neig...,NaN,NaN,You'll have access to the entire space - it's ...,"Very responsive via phone call, text or email.",NaN,4383563,Zander,2012-12-11,"New York, New York, United States","New York City based, currently working as a te...",within a few hours,100%,NaN,f,East Village,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,East Village,Manhattan,New York,NY,10009,New York,US,United States,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""S...",NaN,180.0,1,$0.00,2,1125,3,2018-07-26,2018-10-14,100.0,10.0,9.0,9.0,10.0,10.0,10.0,f,f,moderate,f,f,1,0.89
3,9342478,"Beautiful, airy, light-filled room","Private, spacious, comfortable room in 2-bed f...","Big closet, two big windows, tall ceiling and ...","Private, spacious, comfortable room in 2-bed f...",none,One block from Morgan L stop. Super cool area....,NaN,NaN,NaN,NaN,NaN,10406276,Kathleen,2013-12-03,"New York, New York, United States",Australia

In [7]:
trainC = train.copy()
testC = test.copy()

In [8]:
def sampling_from_dist(table, column):
#     isnull = table[column].isnull()
#     s = table[column].value_counts(normalize=True)
#     sample = np.random.choice(s.index, size=len(table[isnull]),p=s.values)
#     table.loc[table[column].isnull(),column] = sample
#     isnull = table[column].isnull()
#     s = table[column].value_counts(normalize=True)
#     sample = np.random.choice(s.index, size=len(table[isnull]),p=s.values)
    table.loc[table[column].isnull(),column] = table[column].median()
    
def clean(trainC):
    trainC['host_response_rate'] = trainC['host_response_rate'].apply(lambda x: x if pd.isna(x) == True else int(x[:-1]))
    trainC['host_is_superhost'] = trainC['host_is_superhost'].fillna('f').apply(lambda x: 0 if x == 'f' else 1)
    trainC['host_listings_count'] = trainC['host_listings_count'].fillna(0).apply(lambda x: int(x))
    trainC['host_identity_verified'] = trainC['host_identity_verified'].fillna(0).apply(lambda x: 0 if x == 'f' else 1)
#     trainC['bathrooms'] = trainC['bathrooms'].fillna(1.0)
    trainC['extra_people'] = trainC['extra_people'].apply(lambda x: float(x[1:]))
#     trainC['review_scores_accuracy'] = trainC['review_scores_accuracy'].fillna(method = 'ffill')
#     trainC['review_scores_cleanliness'] = trainC['review_scores_cleanliness'].fillna(method = 'ffill')
#     trainC['review_scores_checkin'] = trainC['review_scores_checkin'].fillna(method = 'ffill')
#     trainC['review_scores_communication'] = trainC['review_scores_communication'].fillna(method = 'ffill')
#     trainC['review_scores_location'] = trainC['review_scores_location'].fillna(method = 'ffill')
#     trainC['review_scores_value'] = trainC['review_scores_value'].fillna(method = 'ffill')
#     trainC['reviews_per_month'] = trainC['reviews_per_month'].fillna(method = 'ffill')
    sampling_from_dist(trainC, 'review_scores_accuracy')
    sampling_from_dist(trainC, 'review_scores_cleanliness')
    sampling_from_dist(trainC, 'review_scores_checkin')
    sampling_from_dist(trainC, 'review_scores_communication')
    sampling_from_dist(trainC, 'review_scores_location')
    sampling_from_dist(trainC, 'review_scores_value')
    sampling_from_dist(trainC, 'reviews_per_month')
    sampling_from_dist(trainC, 'bathrooms')
    sampling_from_dist(trainC, 'host_response_rate')
#     sampling_from_dist(trainC, 'review_scores_rating')
    sampling_from_dist(trainC, 'beds')
    trainC['amenities'] = trainC['amenities'].apply(lambda x: len(x.strip('}{').split(',')))
    trainC['instant_bookable'] = trainC['instant_bookable'].fillna(0).apply(lambda x: 0 if x == 'f' else 1)
    trainC['is_business_travel_ready'] = trainC['is_business_travel_ready'].fillna(0).apply(lambda x: 0 if x == 'f' else 1)
    trainC['bedrooms'] = trainC['bedrooms'].fillna(1).apply(lambda x: int(x))
#     trainC['beds'] = trainC['beds'].fillna(1).apply(lambda x: int(x))
    def yearToDay(row):
        started = datetime.datetime.strptime(row,'%Y-%m-%d')
        return started.year * 365 + started.month * 30 + started.day
    trainC['host_since'] = trainC['host_since'].fillna(method = 'ffill')
    trainC['host_since'] = trainC['host_since'].apply(yearToDay)
#     trainC = trainC.drop(columns = ['maximum_nights','state','room_type','host_id','review_scores_rating','zipcode','amenities','country','country_code','city','host_acceptance_rate','square_feet','host_name','name','experiences_offered','notes','host_about','market','first_review','last_review','calculated_host_listings_count','house_rules','host_response_time','host_neighbourhood','require_guest_profile_picture','require_guest_phone_verification','transit','host_since','interaction','access','host_location','neighborhood_overview','host_verifications','host_has_profile_pic','neighbourhood_cleansed','space','summary','description'])
    trainC = trainC.drop(columns = ['review_scores_rating','neighbourhood_group_cleansed','maximum_nights','state','room_type','host_id','zipcode','country','country_code','city','host_acceptance_rate','square_feet','host_name','name','experiences_offered','notes','host_about','market','first_review','last_review','calculated_host_listings_count','house_rules','host_response_time','host_neighbourhood','require_guest_profile_picture','require_guest_phone_verification','transit','interaction','access','host_location','neighborhood_overview','host_verifications','host_has_profile_pic','space','summary','description'])
    return trainC

In [9]:
testC = clean(testC)
testC.head()

,id,host_since,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_cleansed,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,guests_included,extra_people,minimum_nights,number_of_reviews,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,reviews_per_month
0,19307997,736200,100.0,0,1,1,Battery Park City,Condominium,4,2.0,2,2.0,Real Bed,15,1,0.0,2,0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,flexible,0.77
1,20176193,735098,90.0,0,1,0,Bedford-Stuyvesant,Apartment,2,1.0,2,1.0,Real Bed,39,1,0.0,2,22,10.0,9.0,10.0,10.0,10.0,10.0,0,0,flexible,1.48
2,19485371,736017,100.0,0,1,1,Harlem,Apartment,1,1.0,1,1.0,Real Bed,21,1,20.0,1,6,10.0,9.0,10.0,10.0,10.0,10.0,1,0,flexible,0.37
3,13079990,735874,100.0,0,1,0,Brownsville,Apartment,2,1.0,1,1.0,Real Bed,13,1,0.0,1,7,9.0,8.0,10.0,9.0,8.0,9.0,0,0,flexible,0.23
4,22339757,736516,100.0,0,12,0,Bushwick,Apartment,3,2.0,1,2.0,Real Bed,14,2,10.0,2,14,7.0,7.0,9.0,8.0,8.0,8.0,1,0,strict_14_with_grace_period,1.53


In [10]:
trainC = clean(trainC)
trainC.head()

,id,host_since,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_cleansed,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,number_of_reviews,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,reviews_per_month
0,22267382,734482,100.0,0,1,1,Fort Greene,Loft,2,1.0,1,1.0,Real Bed,24,145.0,2,30.0,7,6,10.0,10.0,10.0,10.0,10.0,10.0,1,0,flexible,0.59
1,2473861,735029,100.0,0,4,1,Harlem,House,8,1.0,3,5.0,Real Bed,36,175.0,3,19.0,3,137,9.0,9.0,9.0,9.0,9.0,9.0,1,0,moderate,2.47
2,25079703,734751,100.0,0,1,1,East Village,Apartment,2,1.0,0,1.0,Real Bed,14,180.0,1,0.0,2,3,10.0,9.0,9.0,10.0,10.0,10.0,0,0,moderate,0.89
3,9342478,735108,100.0,0,1,1,Williamsburg,Apartment,1,1.0,1,1.0,Real Bed,19,42.0,1,0.0,3,0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,flexible,0.77
4,4866426,734541,100.0,1,1,1,Boerum Hill,Apartment,2,1.0,1,1.0,Real Bed,19,80.0,1,14.0,1,144,10.0,10.0,10.0,10.0,10.0,10.0,0,0,flexible,3.14


In [11]:
testC.shape

(17337, 28)

In [12]:
trainC.shape

(33538, 29)

In [13]:
# trainC = trainC[(trainC['price'] <= 1650)]

# trainC['neighbourhood_cleansed'] = trainC['neighbourhood_cleansed'].apply(lambda x: np.nan if (x == 'Rossville' or x == 'Little Neck' or x == 'New Brighton' or x == 'Pleasant Plains' or x == 'Arden Heights' or x == 'Bay Terrace, Staten Island' or x == 'Eltingville' or x == 'New Dorp' or x == 'Willowbrook')else x)
# trainC = trainC[~trainC['neighbourhood_cleansed'].isnull()]

In [14]:
X = trainC.drop(columns = ['price','id'])
y = trainC['price']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.4)

In [15]:
X.head()

,host_since,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_cleansed,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,guests_included,extra_people,minimum_nights,number_of_reviews,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,reviews_per_month
0,734482,100.0,0,1,1,Fort Greene,Loft,2,1.0,1,1.0,Real Bed,24,2,30.0,7,6,10.0,10.0,10.0,10.0,10.0,10.0,1,0,flexible,0.59
1,735029,100.0,0,4,1,Harlem,House,8,1.0,3,5.0,Real Bed,36,3,19.0,3,137,9.0,9.0,9.0,9.0,9.0,9.0,1,0,moderate,2.47
2,734751,100.0,0,1,1,East Village,Apartment,2,1.0,0,1.0,Real Bed,14,1,0.0,2,3,10.0,9.0,9.0,10.0,10.0,10.0,0,0,moderate,0.89
3,735108,100.0,0,1,1,Williamsburg,Apartment,1,1.0,1,1.0,Real Bed,19,1,0.0,3,0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,flexible,0.77
4,734541,100.0,1,1,1,Boerum Hill,Apartment,2,1.0,1,1.0,Real Bed,19,1,14.0,1,144,10.0,10.0,10.0,10.0,10.0,10.0,0,0,flexible,3.14


In [16]:
X.columns

Index(['host_since', 'host_response_rate', 'host_is_superhost',
       'host_listings_count', 'host_identity_verified',
       'neighbourhood_cleansed', 'property_type', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'bed_type', 'amenities', 'guests_included',
       'extra_people', 'minimum_nights', 'number_of_reviews',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy', 'reviews_per_month'],
      dtype='object')

In [17]:
X.isnull().sum()

host_since                     0
host_response_rate             0
host_is_superhost              0
host_listings_count            0
host_identity_verified         0
neighbourhood_cleansed         0
property_type                  0
accommodates                   0
bathrooms                      0
bedrooms                       0
beds                           0
bed_type                       0
amenities                      0
guests_included                0
extra_people                   0
minimum_nights                 0
number_of_reviews              0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
instant_bookable               0
is_business_travel_ready       0
cancellation_policy            0
reviews_per_month              0
dtype: int64

In [18]:
preproc = ColumnTransformer(transformers=[
                                          ('encode', OneHotEncoder(handle_unknown = 'ignore'),
                                           ['neighbourhood_cleansed','property_type','cancellation_policy','bed_type']),
#                                           ('kBinReview', KBinsDiscretizer(n_bins = 4,encode = 'ordinal'), ["number_of_reviews",'amenities','host_since']),
#                                         ('binarizerA', Binarizer(threshold = 4), ['accommodates']),
#                                         ('binarizerB', Binarizer(threshold = 2), ['bathrooms']),
#                                         ('kBin', KBinsDiscretizer(n_bins = 3,encode = 'ordinal'),['minimum_nights','extra_people','beds','host_listings_count']),
                                           ('functiontrans', FunctionTransformer(), 
                                            ['host_since', 'host_response_rate', 'host_is_superhost',
                                            'host_listings_count', 'host_identity_verified',
                                            'accommodates', 'bathrooms',
                                           'bedrooms', 'beds', 'amenities', 'guests_included',
                                           'extra_people', 'minimum_nights', 'number_of_reviews',
                                             'review_scores_accuracy',
                                           'review_scores_cleanliness', 'review_scores_checkin',
                                           'review_scores_location', 'review_scores_value', 'instant_bookable',
                                           'is_business_travel_ready', 'reviews_per_month'])
#     ["number_of_reviews",'amenities','host_since','minimum_nights','extra_people','beds','host_listings_count','bathrooms','accommodates','bedrooms','host_response_rate','host_is_superhost','host_identity_verified','guests_included','review_scores_rating','instant_bookable','is_business_travel_ready','reviews_per_month']
                                         ])

# regr1 = RandomForestRegressor(max_depth=6,min_samples_leaf=2)
# regr1 = RandomForestRegressor(criterion='mse',
#                       max_depth=6, min_samples_leaf=3,
#                       min_samples_split=4)
# regr = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth = 6), n_estimators=60)
# regr = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(), n_estimators= 3)
# regr = xgb.XGBRegressor()
regr = xgb.XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=1, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=np.nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1.0, tree_method=None,
             validate_parameters=False, verbosity=None)
# learning_rate=0.05, n_jobs=4, n_estimators = 1000
# regr4 = DecisionTreeRegressor(max_depth = 4)
# vote = VotingRegressor([('rf', regr1),('regr2',regr)])

# # gridsearch
# param = {'max_depth': [1,2,3,4,5,6],
#         'min_samples_leaf': [1,2,3,4,5],
#         'min_samples_split': [2,3,4,5],
#         'criterion': ['mse','mae']}
# rs_grid = RandomizedSearchCV(regr1, param ,scoring = 'neg_root_mean_squared_error')
# X_train_cleaned = preproc.fit_transform(X_train)
# fitted = rs_grid.fit(X_train_cleaned, y_train)
# best_regr = fitted.best_estimator_

pl = Pipeline(steps = [('preprocessor',preproc),
                       ('baseline regression', regr)])
pl.fit(X_train, y_train)
y_train_pred = pl.predict(X_train)
y_train_pred

array([121.12776 ,  66.22076 ,  57.624035, ..., 124.906136, 102.634705,
       210.8622  ], dtype=float32)

In [19]:
pl.score(X_train, y_train)

0.6849605496817309

In [20]:
rms_train = sqrt(mean_squared_error(y_train, y_train_pred))
rms_train

75.27933402758936

In [21]:
#testing on self-made test data

In [22]:
y_test_pred = pl.predict(X_test)
y_test_pred

array([599.9762  ,  80.038666, 178.27412 , ..., 188.49306 , 166.19322 ,
       114.556984], dtype=float32)

In [23]:
pl.score(X_test, y_test)

0.5503652232447775

In [24]:
rms_test = sqrt(mean_squared_error(y_test, y_test_pred))
rms_test

88.18662603681206

In [33]:
pl.fit(X, y) #fitted on the entire X
pred_whole_train = pl.predict(X)

In [34]:
rms_test = sqrt(mean_squared_error(pred_whole_train, y))
rms_test

77.70199396381776

In [35]:
#To turn in

In [36]:
pred = pl.predict(testC.drop(columns = ['id']))
pred

array([351.62198 ,  91.15063 ,  51.83274 , ...,  87.990944,  56.974167,
       198.25874 ], dtype=float32)

In [37]:
len(pred)

17337

In [38]:
final = pd.DataFrame({'Id': testC['id'], 'Predicted': pred})
final.head()

,Id,Predicted
0,19307997,351.621979
1,20176193,91.150627
2,19485371,51.832741
3,13079990,71.269058
4,22339757,54.142273


In [39]:
final.to_csv('final.csv', index = False)